# Explora aquí

Se recomienda utilizar este cuaderno con fines de exploración.

In [21]:
pip install pandas requests lxml

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import re
from bs4 import BeautifulSoup
import requests
import time
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

✅ pandas listo: 3.0.0


In [23]:
url = "https://en.wikipedia.org/wiki/List_of_most-streamed_songs_on_Spotify"
print("✅ Paso 1 - Librerias listas + URL definida")


# -----------------------------
# CELDA 2 - Inspeccionar la pagina (requests -> soup)
# -----------------------------
resp = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
print("✅ Paso 2 - Status code:", resp.status_code)

soup = BeautifulSoup(resp.text, "lxml")
print("✅ Paso 2 - Title:", soup.title)

# mini inspeccion (corta pero demuestra exploracion)
print("✅ Paso 2 - Preview texto (400 chars)")



✅ Paso 1 - Librerias listas + URL definida
✅ Paso 2 - Status code: 200


✅ Paso 2 - Title: <title>List of Spotify streaming records - Wikipedia</title>
✅ Paso 2 - Preview texto (400 chars)


In [27]:
# CELDA 3 - Buscar tablas y ver headers (corto)
tablas = soup.find_all("table")
print("✅ Paso 3 - Tablas encontradas:", len(tablas))

def normalizar(x):
    x = str(x).strip().lower().replace("\xa0", " ")
    x = re.sub(r"\s+", " ", x)
    return x

def headers_text(t):
    ths = t.find_all("th")
    txt = BeautifulSoup(str(ths), "lxml").get_text(" ")
    return normalizar(txt)

# Preview SOLO de las primeras 15 para no llenar la pantalla
for i, t in enumerate(tablas[:15]):
    print(f"- tabla {i}: {headers_text(t)[:140]}")


✅ Paso 3 - Tablas encontradas: 26
- tabla 0: [ rank , song , artist(s) , streams (billions) , release date , ref. , " blinding lights " , " shape of you " , " sweater weather " , " star
- tabla 1: [ rank , artist , top 100 , top 10 , 3 billion+ streams , 4 billion+ streams , 5 billion+ streams , total songs , lead , feat. , lead , feat
- tabla 2: [ no. , song , artist(s) , streams (billions) , release date , date achieved , days held , ref. , notes , as of 12 february 2026 ]
- tabla 3: []
- tabla 4: [ song [ 116 ] , artist(s) , release date , issue date , wks. , avs. , " starboy " , " shape of you " ‡ , " humble " , " despacito (remix) "
- tabla 5: [ year , artist and song ranking , 1 , 2 , 3 , 4 , 5 ]
- tabla 6: [ rank , song , artist(s) , streams , date published , date achieved , ref. , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 1
- tabla 7: [ rank , song , artist(s) , streams , date published , date achieved , ref. , 1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 1

In [28]:
# CELDA 4 - Elegir nuestra tabla (auto)
indice_tabla = None
for i, t in enumerate(tablas):
    h = headers_text(t)
    if ("rank" in h) and ("song" in h) and ("stream" in h):
        indice_tabla = i
        break

print("✅ Paso 4 - indice_tabla detectado:", indice_tabla)

if indice_tabla is None:
    raise ValueError("No se detecto la tabla objetivo. Mira el preview de headers y elige un indice manual.")

tabla_obj = tablas[indice_tabla]
print("✅ Paso 4 - headers (preview):", headers_text(tabla_obj)[:200])


✅ Paso 4 - indice_tabla detectado: 0
✅ Paso 4 - headers (preview): [ rank , song , artist(s) , streams (billions) , release date , ref. , " blinding lights " , " shape of you " , " sweater weather " , " starboy " , " as it was " , " someone you loved " , " sunflower 


In [41]:
# CELDA 5 - Extraer tabla (tr/th/td) y armar df_raw
def texto_celda(tag):
    return tag.get_text(" ", strip=True).replace("\xa0", " ")

rows = tabla_obj.find_all("tr")

# headers
headers = []
for tr in rows[:8]:
    ths = tr.find_all("th")
    if ths:
        headers = [texto_celda(th) for th in ths]
        break

# data
data = []
for tr in rows:
    # saltar la fila de headers (la que no tiene ningun td)
    if tr.find("td") is None:
        continue

    # tomar celdas en orden: th + td (aqui esta la clave)
    celdas = tr.find_all(["th", "td"])
    row = [texto_celda(c) for c in celdas]
    data.append(row)

# ajustar headers y filas
if len(headers) < max_cols: # type: ignore
    headers = headers + [f"col_{i}" for i in range(len(headers), max_cols)] # pyright: ignore[reportUndefinedVariable]
else:
    headers = headers[:max_cols] # type: ignore

data_fix = []
for r in data:
    if len(r) < max_cols: # type: ignore
        r = r + [None] * (max_cols - len(r)) # type: ignore
    data_fix.append(r[:max_cols]) # type: ignore

df_raw = pd.DataFrame(data_fix, columns=headers)

print("✅ Paso 5 - df_raw shape:", df_raw.shape)
print("✅ Paso 5 - columnas:", list(df_raw.columns))
df_raw.head(10)


✅ Paso 5 - df_raw shape: (100, 6)
✅ Paso 5 - columnas: ['Rank', 'Song', 'Artist(s)', 'Streams (billions)', 'Release date', 'Ref.']


,Rank,Song,Artist(s),Streams (billions),Release date,Ref.
0,1,""" Blinding Lights """,The Weeknd,5.279,29 November 2019,[ 1 ]
1,2,""" Shape of You """,Ed Sheeran,4.771,6 January 2017,[ 2 ]
2,3,""" Sweater Weather """,The Neighbourhood,4.384,3 December 2012,[ 3 ]
3,4,""" Starboy """,The Weeknd and Daft Punk,4.367,21 September 2016,[ 4 ]
4,5,""" As It Was """,Harry Styles,4.250,1 April 2022,[ 5 ]
5,6,""" Someone You Loved """,Lewis Capaldi,4.225,8 November 2018,[ 6 ]
6,7,""" Sunflower """,Post Malone and Swae Lee,4.122,18 October 2018,[ 7 ]
7,8,""" One Dance """,Drake with Wizkid and Kyla,4.040,5 April 2016,[ 8 ]
8,9,""" Perfect """,Ed Sheeran,3.830,3 March 2017,[ 9 ]
9,10,""" Stay """,The Kid LAROI and Justin Bieber,3.799,9 July 2021,[ 10 ]


In [34]:
def normalizar(x):
    x = str(x).strip().lower().replace("\xa0", " ")
    return re.sub(r"\s+", " ", x)

cols_norm = {c: normalizar(c) for c in df_raw.columns}

col_rank = col_song = col_artist = col_streams = col_release = col_ref = None
for c, n in cols_norm.items():
    if n == "rank":
        col_rank = c
    elif "song" in n or "title" in n:
        col_song = c
    elif "artist" in n:
        col_artist = c
    elif "stream" in n:
        col_streams = c
    elif ("release" in n and ("date" in n or "year" in n)) or n in ("year", "released"):
        col_release = c
    elif "ref" in n or "reference" in n:
        col_ref = c

# validar (release y ref a veces cambian, pero rank/song/artist/streams deben estar)
criticas = [col_rank, col_song, col_artist, col_streams]
if any(x is None for x in criticas):
    raise ValueError(f"Fallo mapeo columnas. cols_norm={cols_norm}")

# seleccionar columnas (si falta ref, la creamos)
cols_sel = [col_rank, col_song, col_artist, col_streams]
if col_release is not None:
    cols_sel.append(col_release)
else:
    df_raw["Release date"] = np.nan
    col_release = "Release date"
    cols_sel.append(col_release)

if col_ref is not None:
    cols_sel.append(col_ref)
else:
    df_raw["Ref."] = np.nan
    col_ref = "Ref."
    cols_sel.append(col_ref)

df = df_raw[cols_sel].copy()
df.head(10)


,Rank,Song,Artist(s),Streams (billions),Release date,Ref.
0,1,""" Blinding Lights """,The Weeknd,5.279,29 November 2019,[ 1 ]
1,2,""" Shape of You """,Ed Sheeran,4.771,6 January 2017,[ 2 ]
2,3,""" Sweater Weather """,The Neighbourhood,4.384,3 December 2012,[ 3 ]
3,4,""" Starboy """,The Weeknd and Daft Punk,4.367,21 September 2016,[ 4 ]
4,5,""" As It Was """,Harry Styles,4.250,1 April 2022,[ 5 ]
5,6,""" Someone You Loved """,Lewis Capaldi,4.225,8 November 2018,[ 6 ]
6,7,""" Sunflower """,Post Malone and Swae Lee,4.122,18 October 2018,[ 7 ]
7,8,""" One Dance """,Drake with Wizkid and Kyla,4.040,5 April 2016,[ 8 ]
8,9,""" Perfect """,Ed Sheeran,3.830,3 March 2017,[ 9 ]
9,10,""" Stay """,The Kid LAROI and Justin Bieber,3.799,9 July 2021,[ 10 ]


In [ ]:
def clean_text(s):
    s = s.astype(str)
    s = s.str.replace("\xa0", " ", regex=False)
    s = s.str.replace(r"\[\s*\d+\s*\]", "", regex=True)
    s = s.str.replace(r"\s+", " ", regex=True).str.strip()
    s = s.replace({"nan": np.nan, "none": np.nan, "": np.nan})
    return s

# texto
df[col_song] = clean_text(df[col_song])
df[col_artist] = clean_text(df[col_artist])
df[col_ref] = clean_text(df[col_ref])

# rank
df[col_rank] = pd.to_numeric(df[col_rank], errors="coerce").astype("Int64")

# streams (quita comas y convierte)
df[col_streams] = df[col_streams].astype(str).str.replace(",", "", regex=False).str.strip()
df[col_streams] = pd.to_numeric(df[col_streams], errors="coerce")

# release date
df[col_release] = pd.to_datetime(df[col_release], errors="coerce").dt.date

# filas validas
df = df[df[col_rank].notna()].reset_index(drop=True)

# top 100
df_100 = df.sort_values(col_rank).head(100).reset_index(drop=True)

pd.set_option("display.width", 300)
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)

print(df_100.head(10))

   Rank                   Song                        Artist(s)  Streams (billions) Release date Ref.
0     1    " Blinding Lights "                       The Weeknd               5.279   2019-11-29  NaN
1     2       " Shape of You "                       Ed Sheeran               4.771   2017-01-06  NaN
2     3    " Sweater Weather "                The Neighbourhood               4.384   2012-12-03  NaN
3     4            " Starboy "         The Weeknd and Daft Punk               4.367   2016-09-21  NaN
4     5          " As It Was "                     Harry Styles               4.250   2022-04-01  NaN
5     6  " Someone You Loved "                    Lewis Capaldi               4.225   2018-11-08  NaN
6     7          " Sunflower "         Post Malone and Swae Lee               4.122   2018-10-18  NaN
7     8          " One Dance "       Drake with Wizkid and Kyla               4.040   2016-04-05  NaN
8     9            " Perfect "                       Ed Sheeran               3.83